In [ ]:
!pip install -q -U transformers accelerate bitsandbytes torch

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
model_name="unsloth/Qwen3-1.7B-bnb-4bit"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    dtype="auto",
    device_map="auto"
)

In [ ]:
prompt = "Why is TP53 an important therapeutic target?"
messages = [
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=True # Switches between thinking and non-thinking modes. Default is True.
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

# conduct text completion
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=32768
)
output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist()

# parsing thinking content
try:
    # rindex finding 151668 (</think>)
    index = len(output_ids) - output_ids[::-1].index(151668)
except ValueError:
    index = 0

thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")

# print("thinking content:", thinking_content)
print("content:", content)

content: The **TP53** gene is a critical tumor suppressor gene that plays a central role in maintaining genomic stability, regulating the cell cycle, and initiating apoptosis in response to DNA damage. Its dysfunction is a hallmark of many cancers, making it a pivotal therapeutic target. Below is a structured explanation of why TP53 is an important therapeutic target:

---

### **1. Role of TP53 in Cell Cycle and Apoptosis**
- **Cell Cycle Regulation**: TP53 acts as a "guardian of the genome" by monitoring DNA integrity. When DNA damage occurs, TP53 activates the **p53 pathway**, which halts the cell cycle (via **G1 checkpoint**) and promotes DNA repair. If the damage is irreparable, TP53 triggers **apoptosis** (programmed cell death) to eliminate damaged cells.
- **Apoptosis Induction**: TP53 is a key mediator of apoptosis. Mutant TP53 (e.g., in *TP53* mutations) fails to induce apoptosis, leading to uncontrolled cell proliferation and cancer.

---

### **2. TP53 Mutations in Cancer**

In [ ]:
from datasets import load_dataset

raw_data = load_dataset("json", data_files="drug_target_discovery_200.json")
print(raw_data)

In [ ]:


raw_data["train"][0]



{'prompt': 'Describe how HER2 contributes to cellular signaling in disease.',
 'completion': 'HER2 is a growth factor receptor amplified in breast cancers; targeted by trastuzumab.'}

In [ ]:

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(
    model_name
)

def preprocess(sample):
    sample = sample["prompt"] + "\n" + sample["completion"]

    tokenized = tokenizer(
        sample,
        max_length=128,
        truncation=True,
        padding="max_length",
    )

    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

data = raw_data.map(preprocess)


In [ ]:
print(data["train"][0])

{'prompt': 'Describe how HER2 contributes to cellular signaling in disease.', 'completion': 'HER2 is a growth factor receptor amplified in breast cancers; targeted by trastuzumab.', 'input_ids': [151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 74785, 1246, 15645, 17, 4297

In [ ]:
from peft import LoraConfig, get_peft_model, TaskType
from transformers import AutoModelForCausalLM
import torch

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    dtype = torch.float16
)

lora_config = LoraConfig(
    task_type = TaskType.CAUSAL_LM,
    target_modules = ["q_proj", "k_proj", "v_proj"]
)

model = get_peft_model(model, lora_config)

# Explicitly move the model to the CUDA device
model.to("cuda")

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen3ForCausalLM(
      (model): Qwen3Model(
        (embed_tokens): Embedding(151936, 2048, padding_idx=151654)
        (layers): ModuleList(
          (0-27): 28 x Qwen3DecoderLayer(
            (self_attn): Qwen3Attention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    num_train_epochs=10,
    learning_rate=0.001,
    logging_steps=25,
    report_to="none" # Disable wandb logging
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=data["train"]
)

trainer.train()

Step,Training Loss
25,13.634900
50,13.364800
75,13.349000
100,13.349100
125,13.348100
150,13.347400
175,13.346700
200,13.346300
225,13.346000
250,13.345500


TrainOutput(global_step=250, training_loss=13.377767944335938, metrics={'train_runtime': 668.9913, 'train_samples_per_second': 2.99, 'train_steps_per_second': 0.374, 'total_flos': 2168377049088000.0, 'train_loss': 13.377767944335938, 'epoch': 10.0})

In [ ]:
trainer.save_model("Qwen-1.7B-MedQA-LoRA")
tokenizer.save_pretrained("Qwen-1.7B-MedQA-LoRA")

('Qwen-1.7B-MedQA-LoRA/tokenizer_config.json',
 'Qwen-1.7B-MedQA-LoRA/special_tokens_map.json',
 'Qwen-1.7B-MedQA-LoRA/chat_template.jinja',
 'Qwen-1.7B-MedQA-LoRA/vocab.json',
 'Qwen-1.7B-MedQA-LoRA/merges.txt',
 'Qwen-1.7B-MedQA-LoRA/added_tokens.json',
 'Qwen-1.7B-MedQA-LoRA/tokenizer.json')

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel, PeftConfig

path = "/content/Qwen-1.7B-MedQA-LoRA"

config = PeftConfig.from_pretrained(path)
base = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, trust_remote_code=True)
model = PeftModel.from_pretrained(base, path)

tokenizer = AutoTokenizer.from_pretrained(path, trust_remote_code=True)



In [ ]:
inputs = tokenizer( "Why is TP53 an important therapeutic target?", return_tensors="pt").to(model.device)

output = model.generate(
    input_ids=inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    max_new_tokens=100 # Reduced for faster generation, adjust as needed
)

print(tokenizer.decode(output[0]))

Why is TP53 an important therapeutic target? TP53 is a tumor suppressor protein that regulates DNA repair and apoptosis; mutations lead to uncontrolled cell growth. BRAF is a MAPK pathway kinase; V600E mutation causes melanoma and other cancers. TP53 is a key player in apoptosis and DNA repair; mutations lead to uncontrolled cell growth.
TP53 is a critical tumor suppressor protein that regulates apoptosis and DNA repair. Mutations lead to uncontrolled cell growth and resistance to apoptosis. BRAF is


## Install huggingface hub

Install the necessary library to interact with the Hugging Face Hub.


**Reasoning**:
The subtask is to install the `huggingface_hub` library. The `pip install` command is used to install Python packages.



In [ ]:
!pip install -q huggingface_hub

## Login to hugging face hub

### Subtask:
Authenticate with your Hugging Face account.


**Reasoning**:
Import the `notebook_login` function and call it to authenticate with Hugging Face.



In [ ]:
from huggingface_hub import notebook_login

notebook_login()

## Push model and tokenizer

### Subtask:
Push the fine-tuned model and tokenizer to the Hugging Face Hub.


**Reasoning**:
Push the fine-tuned model and tokenizer to the Hugging Face Hub using the `push_to_hub` method.



In [ ]:
model.push_to_hub("Qwen-1.7B-MedQA-LoRA")
tokenizer.push_to_hub("Qwen-1.7B-MedQA-LoRA")

In [ ]:
import gradio as gr
from transformers import pipeline


model_id = "Sabarinath-K/Qwen-1.7B-MedQA-LoRA"

classifier = pipeline("text-generation", model=model_id)

def inference(text):
    output = classifier(text)[0]
    print("Classifier output:", output) # Add this line to inspect the output
    return output['generated_text']

iface = gr.Interface(
    fn=inference,
    inputs="text",
    outputs="text",
    title="Qwen-1.7B-MedQA-LoRA"
)

iface.launch()

Device set to use cuda:0


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ab6c09b7dee14b959e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
